# APIs Lab
In this lab we will practice using APIs to retrieve and store data.

In [1]:
# Imports at the top
import json
import urllib
import pandas as pd
import numpy as np
import requests
import json
import re

## Exercise 1: Get Data From Sheetsu

[Sheetsu](https://sheetsu.com/) is an online service that allows you to access any Google spreadsheet from an API. This can be a very powerful way to share a dataset with colleagues as well as to create a mini centralized data storage, that is simpler to edit than a database.

A Google Spreadsheet with Wine data can be found [here](https://docs.google.com/spreadsheets/d/1mZ3Otr5AV4v8WwvLOAvWf3VLxDa-IeJ1AVTEuavJqeo/).

It can be accessed through sheetsu API at this endpoint: https://sheetsu.com/apis/v1.0/dab55afd

Questions:

1. Use the requests library to access the document. Inspect the response text. What kind of data is it?
> Answer: it's a json string
- Check the status code of the response object. What code is it?
> 200
- Use the appropriate libraries and read functions to read the response into a Pandas Dataframe
> Possible answers include: pd.read_json and json.loads + pd.Dataframe
- Once you've imported the data into a dataframe, check the value of the 5th line: what's the price?
> 6

In [44]:
# You can either post or get info from this API
api_base_url = 'https://sheetsu.com/apis/v1.0/dab55afd'

In [45]:
# What kind of data is this returning?
api_response = requests.get(api_base_url)
api_response.text[:100]

u'[{"Color":"W","Region":"Portugal","Country":"Portugal","Vintage":"2013","Vinyard":"Vinho Verde","Nam'

In [46]:
reponse = json.loads(api_response.text)

In [47]:
type(reponse)

list

In [48]:
reponse[0]

{u'Color': u'W',
 u'Consumed In': u'2015',
 u'Country': u'Portugal',
 u'Grape': u'',
 u'Name': u'',
 u'Price': u'',
 u'Region': u'Portugal',
 u'Score': u'4',
 u'Vintage': u'2013',
 u'Vinyard': u'Vinho Verde'}

In [7]:
api_response.status_code

200

Information on [status codes](http://www.w3.org/Protocols/rfc2616/rfc2616-sec10.html).

#### Lets read the data into a DataFrame!

In [8]:
wine_df = pd.DataFrame(reponse)
wine_df.head()

,Color,Consumed In,Country,Grape,Name,Price,Region,Score,Vintage,Vinyard
0,W,2015,Portugal,,,,Portugal,4,2013,Vinho Verde
1,W,2015,France,,,17.8,France,3,2013,Peyruchet
2,W,2015,Oregon,,,20,Oregon,3,2013,Abacela
3,W,2015,Spain,chardonay,,7,Spain,2.5,2012,Ochoa
4,R,2015,US,"chiraz, cab",Spice Trader,6,,3,2012,Heartland


#### Pandas has great functions. We could just do it this way

This sometimes works, but the data may need adjusting

In [9]:
wine_df = pd.read_json(api_response.text)
wine_df.head(2)

,Color,Consumed In,Country,Grape,Name,Price,Region,Score,Vintage,Vinyard
0,W,2015,Portugal,,,,Portugal,4,2013,Vinho Verde
1,W,2015,France,,,17.8,France,3,2013,Peyruchet


### Exercise 2: Post Data to Sheetsu
Now that we've learned how to read data, it'd be great if we could also write data. For this we will need to use a _POST_ request.

1. Use the post command to add the following data to the spreadsheet:
- What status did you get? How can you check that you actually added the data correctly?
> Answer: send a get request and check the last line added
- In this exercise, your classmates are adding data to the same spreadsheet. What happens because of this? Is it a problem? How could you mitigate it?
> There will be many duplicate lines on the spreadsheet. One way to mitigate this would be through permission, another would be to insert at a specific position, so that the line is overwritten at each time.


In [10]:
post_data = {
'Grape' : ''
, 'Name' : 'My wonderful wine'
, 'Color' : 'R'
, 'Country' : 'US'
, 'Region' : 'Sonoma'
, 'Vinyard' : ''
, 'Score' : '10'
, 'Consumed In' : '2015'
, 'Vintage' : '1973'
, 'Price' : '200'
}

In [11]:
requests.post(api_base_url, data=post_data)

<Response [201]>

Information on [status codes](http://www.w3.org/Protocols/rfc2616/rfc2616-sec10.html).

## Exercise 3: Data munging

Get back to the dataframe you've created in the beginning. Let's do some data munging:

1. Search for missing data
    - Is there any missing data? How do you deal with it?
    - Is there any data you can just remove?
- Summarize the data 
    - Try using describe, min, max, mean, var

In [12]:
wine_df.head(1)

,Color,Consumed In,Country,Grape,Name,Price,Region,Score,Vintage,Vinyard
0,W,2015,Portugal,,,,Portugal,4,2013,Vinho Verde


In [13]:
wine_df = wine_df.replace('', np.nan)

In [14]:
wine_df.head(1)

,Color,Consumed In,Country,Grape,Name,Price,Region,Score,Vintage,Vinyard
0,W,2015,Portugal,NaN,NaN,NaN,Portugal,4,2013,Vinho Verde


In [15]:
wine_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 101 entries, 0 to 100
Data columns (total 10 columns):
Color          101 non-null object
Consumed In    101 non-null object
Country        99 non-null object
Grape          22 non-null object
Name           92 non-null object
Price          95 non-null object
Region         100 non-null object
Score          100 non-null object
Vintage        101 non-null object
Vinyard        35 non-null object
dtypes: object(10)
memory usage: 8.7+ KB


In [17]:
wine_df.describe()

,Color,Consumed In,Country,Grape,Name,Price,Region,Score,Vintage,Vinyard
count,101,101,99,22,92,95,100,100,101,35
unique,5,9,7,16,36,20,22,10,11,30
top,R,2015,US,sauvignon blanc,My wonderful wine,200,Sonoma,10,1973,Top of the line wine
freq,81,82,81,4,45,68,65,67,64,3


In [18]:
wine_df.head()

,Color,Consumed In,Country,Grape,Name,Price,Region,Score,Vintage,Vinyard
0,W,2015,Portugal,NaN,NaN,NaN,Portugal,4,2013,Vinho Verde
1,W,2015,France,NaN,NaN,17.8,France,3,2013,Peyruchet
2,W,2015,Oregon,NaN,NaN,20,Oregon,3,2013,Abacela
3,W,2015,Spain,chardonay,NaN,7,Spain,2.5,2012,Ochoa
4,R,2015,US,"chiraz, cab",Spice Trader,6,NaN,3,2012,Heartland


## Exercise 4: IMDB Movies

Sometimes an API doesn't provide all the information we would like to get and we need to be creative.
Here we will use a combination of scraping and API calls to investigate the ratings and gross earnings of famous movies.

## 4.a Get top movies

The Internet Movie Database contains data about movies. Unfortunately it does not have a public API.

The page http://www.imdb.com/chart/top contains the list of the top 250 movies of all times. Retrieve the page using the requests library and then parse the html to obtain a list of the `movie_ids` for these movies. You can parse it with regular expression or using a library like `BeautifulSoup`.

**Hint:** movie_ids look like this: `tt2582802`

In [ ]:
def get_top_250():
    response = requests.get('http://www.imdb.com/chart/top')
    html = response.text
    entries = re.findall("<a href.*?/title/(.*?)/", html) 
    return list(set(entries))

* .	Matches any single character except newline.
* `*`	Matches 0 or more occurrences of preceding expression.
* ?	Matches 0 or 1 occurrence of preceding expression.

In [51]:
entries = get_top_250()

In [52]:
len(entries)

250

In [53]:
entries[0]

u'tt2582802'

## 4.b Get top movies data

Although the Internet Movie Database does not have a public API, an open API exists at http://www.omdbapi.com.

Use this API to retrieve information about each of the 250 movies you have extracted in the previous step.
1. Check the documentation of omdbapi.com to learn how to request movie data by id
- Define a function that returns a python object with all the information for a given id
- Iterate on all the IDs and store the results in a list of such objects
- Create a Pandas Dataframe from the list

In [54]:
def get_entry(entry):
    res = requests.get('http://www.omdbapi.com/?i='+entry)
    if res.status_code != 200:
        print entry, res.status_code
    else:
        print '.',
    try:
        j = json.loads(res.text)
    except ValueError:
        j = None
    return j

In [55]:
entries_dict_list = [get_entry(e) for e in entries]

. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .


In [56]:
len(entries_dict_list)

250

In [57]:
df = pd.DataFrame(entries_dict_list)

In [40]:
df.head(3)

,Actors,Awards,Country,Director,Genre,Language,Metascore,Plot,Poster,Rated,Released,Response,Runtime,Title,Type,Writer,Year,imdbID,imdbRating,imdbVotes
0,"Miles Teller, J.K. Simmons, Paul Reiser, Melis...",Won 3 Oscars. Another 87 wins & 131 nominations.,USA,Damien Chazelle,"Drama, Music",English,88,A promising young drummer enrolls at a cut-thr...,http://ia.media-imdb.com/images/M/MV5BMTU4OTQ3...,R,15 Oct 2014,True,107 min,Whiplash,movie,Damien Chazelle,2014,tt2582802,8.5,"396,001"
1,"Toshirô Mifune, Takashi Shimura, Keiko Tsushim...",Nominated for 2 Oscars. Another 5 wins & 6 nom...,Japan,Akira Kurosawa,"Action, Adventure, Drama",Japanese,99,A poor village under attack by bandits recruit...,http://ia.media-imdb.com/images/M/MV5BMTc5MDY1...,UNRATED,19 Nov 1956,True,207 min,Seven Samurai,movie,"Akira Kurosawa (screenplay), Shinobu Hashimoto...",1954,tt0047478,8.7,"226,364"
2,"Harrison Ford, Karen Allen, Paul Freeman, Rona...",Won 4 Oscars. Another 29 wins & 23 nominations.,USA,Steven Spielberg,"Action, Adventure","English, German, Hebrew, Spanish, Arabic, Nepali",85,Archaeologist and adventurer Indiana Jones is ...,http://ia.media-imdb.com/images/M/MV5BMjA0ODEz...,PG,12 Jun 1981,True,115 min,Raiders of the Lost Ark,movie,"Lawrence Kasdan (screenplay), George Lucas (st...",1981,tt0082971,8.5,"653,557"


In [41]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 250 entries, 0 to 249
Data columns (total 20 columns):
Actors        250 non-null object
Awards        250 non-null object
Country       250 non-null object
Director      250 non-null object
Genre         250 non-null object
Language      250 non-null object
Metascore     250 non-null object
Plot          250 non-null object
Poster        250 non-null object
Rated         250 non-null object
Released      250 non-null object
Response      250 non-null object
Runtime       250 non-null object
Title         250 non-null object
Type          250 non-null object
Writer        250 non-null object
Year          250 non-null object
imdbID        250 non-null object
imdbRating    250 non-null object
imdbVotes     250 non-null object
dtypes: object(20)
memory usage: 39.1+ KB


## 4.c Data munging

1. Now that you have movie information and gross revenue information, let's clean the two datasets.
- Check if there are null values. Be careful they may appear to be valid strings.
- Convert the columns to the appropriate formats. In particular handle:
    - Released
    - Runtime
    - year
    - imdbRating
    - imdbVotes
- Merge the data from the two datasets into a single one

In [72]:
df = df.replace('N/A', np.nan)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 250 entries, 0 to 249
Columns: 855 entries, Actors to actor: álvaro guerrero
dtypes: int64(835), object(20)
memory usage: 1.6+ MB


In [73]:
df.Released = pd.to_datetime(df.Released)

In [74]:
def intminutes(x):
    y = x.replace('min', '').strip()
    return int(y)

df.Runtime = df.Runtime.apply(intminutes)

In [75]:
df.Year = df.Year.astype(int)

In [76]:
df.imdbRating = df.imdbRating.astype(float)

In [77]:
def intvotes(x):
    y = x.replace(',', '').strip()
    return int(y)
df.imdbVotes = df.imdbVotes.apply(intvotes)

In [78]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 250 entries, 0 to 249
Columns: 855 entries, Actors to actor: álvaro guerrero
dtypes: datetime64[ns](1), float64(1), int64(838), object(15)
memory usage: 1.6+ MB


In [79]:
df = pd.merge(df, df1)

In [80]:
df.head()

,Actors,Awards,Country,Director,Genre,Language,Metascore,Plot,Poster,Rated,...,actor: xolani mali,actor: yacef saadi,actor: yoshiko shinohara,actor: yukiko shimazaki,actor: yves montand,actor: yôko tsukasa,actor: zach grenier,actor: zoe saldana,actor: álvaro guerrero,Gross
0,"Miles Teller, J.K. Simmons, Paul Reiser, Melis...",Won 3 Oscars. Another 87 wins & 131 nominations.,USA,Damien Chazelle,"Drama, Music",English,88,A promising young drummer enrolls at a cut-thr...,http://ia.media-imdb.com/images/M/MV5BMTU4OTQ3...,R,...,0,0,0,0,0,0,0,0,0,13092000.0
1,"Toshirô Mifune, Takashi Shimura, Keiko Tsushim...",Nominated for 2 Oscars. Another 5 wins & 6 nom...,Japan,Akira Kurosawa,"Action, Adventure, Drama",Japanese,99,A poor village under attack by bandits recruit...,http://ia.media-imdb.com/images/M/MV5BMTc5MDY1...,UNRATED,...,0,0,0,1,0,0,0,0,0,269061.0
2,"Harrison Ford, Karen Allen, Paul Freeman, Rona...",Won 4 Oscars. Another 29 wins & 23 nominations.,USA,Steven Spielberg,"Action, Adventure","English, German, Hebrew, Spanish, Arabic, Nepali",85,Archaeologist and adventurer Indiana Jones is ...,http://ia.media-imdb.com/images/M/MV5BMjA0ODEz...,PG,...,0,0,0,0,0,0,0,0,0,242374454.0
3,"William Holden, Alec Guinness, Jack Hawkins, S...",Won 7 Oscars. Another 23 wins & 7 nominations.,"UK, USA",David Lean,"Adventure, Drama, War","English, Japanese, Thai",NaN,After settling his differences with a Japanese...,http://ia.media-imdb.com/images/M/MV5BMTc2NzA0...,PG,...,0,0,0,0,0,0,0,0,0,27200000.0
4,"Robert Downey Jr., Chris Evans, Mark Ruffalo, ...",Nominated for 1 Oscar. Another 36 wins & 78 no...,USA,Joss Whedon,"Action, Adventure, Sci-Fi","English, Russian",69,Earth's mightiest heroes must come together an...,http://ia.media-imdb.com/images/M/MV5BMTk2NTI1...,PG-13,...,0,0,0,0,0,0,0,0,0,623279547.0


## 4.d Text vectorization

There are several columns in the data that contain a comma separated list of items, for example the Genre column and the Actors column. Let's transform those to binary columns using the count vectorizer from scikit learn.

Append these columns to the merged dataframe.

**Hint:** In order to get the actors name right, you'll have to modify the `token_pattern` in the `CountVectorizer`.

In [81]:
from sklearn.feature_extraction.text import CountVectorizer

In [82]:
cv = CountVectorizer()
data = cv.fit_transform(df.Genre).todense()
columns = ['genre_'+c for c in cv.get_feature_names()]
genredf = pd.DataFrame(data, columns=columns)
genredf.head()

,genre_action,genre_adventure,genre_animation,genre_biography,genre_comedy,genre_crime,genre_drama,genre_family,genre_fantasy,genre_fi,...,genre_music,genre_musical,genre_mystery,genre_noir,genre_romance,genre_sci,genre_sport,genre_thriller,genre_war,genre_western
0,0,0,0,0,0,0,1,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1,1,1,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,1,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0
4,1,1,0,0,0,0,0,0,0,1,...,0,0,0,0,0,1,0,0,0,0


In [83]:
df = pd.concat([df, genredf], axis = 1)

In [84]:
cv = CountVectorizer(token_pattern=u'(?u)\\w+\.?\\w?\.? \\w+')
data = cv.fit_transform(df.Actors).todense()
columns = ['actor: '+c for c in cv.get_feature_names()]
actorsdf = pd.DataFrame(data, columns=columns)
actorsdf.head()

,actor: aamir khan,actor: aaron eckhart,actor: abdel ahmed,actor: adam baldwin,actor: adam driver,actor: adolphe menjou,actor: adrien brody,actor: agnes moorehead,actor: ahney her,actor: akemi yamaguchi,...,actor: woody harrelson,actor: xolani mali,actor: yacef saadi,actor: yoshiko shinohara,actor: yukiko shimazaki,actor: yves montand,actor: yôko tsukasa,actor: zach grenier,actor: zoe saldana,actor: álvaro guerrero
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [85]:
df.Actors[0]

u'Miles Teller, J.K. Simmons, Paul Reiser, Melissa Benoist'

In [86]:
actorsdf.loc[0,actorsdf.iloc[0] != 0]

actor: j.k. simmons       1
actor: melissa benoist    1
actor: miles teller       1
actor: paul reiser        1
Name: 0, dtype: int64

In [87]:
df = pd.concat([df, actorsdf], axis = 1)

## Bonus:

1. What are the top 10 grossing movies?
- Who are the 10 actors that appear in the most movies?
- What genre is the oldest movie?


In [88]:
df.columns

Index([                  u'Actors',                   u'Awards',
                        u'Country',                 u'Director',
                          u'Genre',                 u'Language',
                      u'Metascore',                     u'Plot',
                         u'Poster',                    u'Rated',
       ...
         u'actor: woody harrelson',       u'actor: xolani mali',
             u'actor: yacef saadi', u'actor: yoshiko shinohara',
        u'actor: yukiko shimazaki',      u'actor: yves montand',
            u'actor: yôko tsukasa',      u'actor: zach grenier',
             u'actor: zoe saldana',   u'actor: álvaro guerrero'],
      dtype='object', length=1691)

In [89]:
df[['Title','Gross', 'Genre']].sort_values('Gross', ascending = False).head(10)

,Title,Gross,Genre
161,Star Wars: The Force Awakens,936627416.0,"Action, Adventure, Fantasy"
4,The Avengers,623279547.0,"Action, Adventure, Sci-Fi"
39,The Dark Knight,533316061.0,"Action, Adventure, Crime"
235,Star Wars: Episode IV - A New Hope,460935665.0,"Action, Adventure, Fantasy"
160,The Dark Knight Rises,448130642.0,"Action, Adventure, Drama"
50,The Lion King,422783777.0,"Animation, Adventure, Drama"
102,Toy Story 3,414984497.0,"Animation, Adventure, Comedy"
229,Captain America: Civil War,406573683.0,"Action, Adventure, Sci-Fi"
70,Harry Potter and the Deathly Hallows: Part 2,380955619.0,"Adventure, Drama, Fantasy"
9,Finding Nemo,380838870.0,"Animation, Adventure, Comedy"


In [90]:
actorcols = actorsdf.columns

In [91]:
topactors = actorsdf.sum().sort_values(ascending = False).head(10)
topactors

actor: robert de            7
actor: leonardo dicaprio    7
actor: harrison ford        7
actor: tom hanks            6
actor: clint eastwood       6
actor: tom hardy            5
actor: mark hamill          5
actor: al pacino            4
actor: toshirô mifune       4
actor: matt damon           4
dtype: int64

In [95]:
df.sort_values('Released')[['Genre', 'Title', 'Released', 'Gross']].head()

,Genre,Title,Released,Gross
90,"Comedy, Drama, Family",The Kid,1921-02-06,2500000.0
180,"Action, Adventure, Comedy",The General,1927-02-24,NaN
211,"Drama, Sci-Fi",Metropolis,1927-03-13,26435.0
194,"Drama, Romance",Sunrise,1927-11-04,NaN
25,"Comedy, Drama, Romance",City Lights,1931-03-07,NaN
